# V2.2 Personalization Research: User Profile-Based Customization

**Author**: CLI 4 (The Lab)  
**Date**: 2025-10-06  
**Status**: 🔬 Research Phase  
**Parent Feature**: V2.1 Prescriptive Analysis  

---

## 🎯 Research Question

**How can we leverage user's historical match data and role/champion preferences to dynamically customize analysis reports?**

### Core Hypotheses

1. **H1 (Profile-Based Prioritization)**: Users with persistent weaknesses in specific dimensions (e.g., Vision scores consistently low) should receive prioritized suggestions for those dimensions, even if other dimensions had larger gaps in a single match.

2. **H2 (Tone Customization)**: Casual/recreational players prefer encouraging, humorous tone with simpler language, while competitive players prefer concise, data-heavy, critical analysis.

3. **H3 (Role-Specific Suggestions)**: ADC players and Support players should receive different types of Vision suggestions (e.g., ADC: "buy Control Ward before Dragon fight" vs Support: "rotate to ward Baron pit at 18:00").

---

## 📊 Technical Approaches

### Approach A: Prompt Dynamic Injection (User Profile Prefix)

**Concept**: Inject user profile data into LLM prompt to influence suggestion generation.

**Implementation**:
```python
user_profile = {
    "preferred_role": "Jungle",
    "avg_vision_score": 45.2,  # Across last 20 matches
    "team_avg_vision_score": 67.8,
    "persistent_weak_dimension": "Vision",  # Low in 75% of matches
    "skill_level": "intermediate",
    "player_type": "competitive",  # vs "casual"
}

profile_context = f"""
**用户画像**：该用户是一个 {user_profile['preferred_role']} 位置玩家，
视野得分在最近 20 场比赛中持续低于队伍平均水平（平均 {user_profile['avg_vision_score']} 分，
队伍平均 {user_profile['team_avg_vision_score']} 分）。这是他需要优先改进的维度。
"""
```

**Pros**:
- LLM can naturally integrate profile into all suggestions
- No post-processing needed
- Can influence both content and tone

**Cons**:
- Increases input token cost (~50-100 tokens)
- LLM might hallucinate profile details if not careful

---

### Approach B: Post-Processing Filtering & Re-ranking

**Concept**: Generate suggestions using V2.1 baseline, then reorder based on user profile.

**Implementation**:
```python
def rerank_suggestions(
    suggestions: list[V21ImprovementSuggestion],
    user_profile: UserProfile,
) -> list[V21ImprovementSuggestion]:
    """
    Boost priority of suggestions matching persistent weak dimensions.
    """
    def priority_score(suggestion: V21ImprovementSuggestion) -> int:
        base_score = {"critical": 3, "high": 2, "medium": 1}[suggestion.priority]
        
        # Boost if dimension matches persistent weakness
        if suggestion.dimension == user_profile.persistent_weak_dimension:
            base_score += 2
        
        # Boost if role-specific (e.g., Vision for Support/Jungle)
        if (
            suggestion.dimension == "Vision"
            and user_profile.preferred_role in ["Support", "Jungle"]
        ):
            base_score += 1
        
        return base_score
    
    return sorted(suggestions, key=priority_score, reverse=True)
```

**Pros**:
- No change to prompt or LLM input
- Deterministic and testable
- Can apply A/B testing to reranking logic

**Cons**:
- Cannot change suggestion content or tone
- Only affects ordering

---

### Approach C: Tone-Specific Prompt Variants

**Concept**: Maintain 2-3 prompt templates optimized for different player types.

**Implementation**:
```python
TONE_VARIANTS = {
    "competitive": {
        "system_role": "你是一位专业的电竞数据分析师，专注于提供高密度、批判性反馈。",
        "language_style": "简洁、技术性强、避免重复解释",
        "example_action_item": "20分钟前必须放置大龙坑真眼（位置：9500, 5200），延迟回城也要执行。",
    },
    "casual": {
        "system_role": "你是一位友好的游戏教练，通过轻松、鼓励的方式帮助玩家提升。",
        "language_style": "通俗易懂、适度使用比喻、鼓励性语言",
        "example_action_item": "下次打大龙前，记得提前1分钟去河道放个真眼，就像给队友买了个'保险'！",
    },
}
```

**Pros**:
- Dramatically different tone and complexity
- Easy to test with A/B variants

**Cons**:
- Maintenance overhead (2-3 prompt templates)
- Requires user profiling logic to select variant

---

## 🧪 Experiments

### Experiment 1: Profile Injection Impact on Suggestion Relevance

**Setup**:
- Same weak dimensions input (Vision score 62.4, team rank 4/5)
- Compare output with/without user profile injection

**Evaluation Metric**:
- **Relevance Score**: Does the suggestion acknowledge user's persistent weakness?
- **Token Cost Delta**: How many extra tokens does profile injection add?


In [ ]:
# Experiment 1: Mock LLM Responses

# Baseline V2.1 Output (No Profile)
baseline_suggestion = {
    "suggestion_id": "Vision_1456000",
    "dimension": "Vision",
    "issue_identified": "视野控制弱于队友（评分62.4，队伍排名第4），导致24:16大龙被敌方偷取",
    "action_item": (
        "在大龙刷新前60秒（游戏时间20分钟后），"
        "优先购买并放置真眼在大龙坑上方河道草丛。"
    ),
    "expected_outcome": "提升团队对大龙区域的视野控制率从0%到至少50%。",
    "priority": "critical",
}

# With Profile Injection
personalized_suggestion = {
    "suggestion_id": "Vision_1456000",
    "dimension": "Vision",
    "issue_identified": (
        "作为打野位置，你的视野得分在最近20场比赛中持续偏低（平均45.2分，队伍平均67.8分）。"
        "本场比赛这一趋势继续，评分62.4（排名第4），直接导致24:16大龙失误。"
    ),
    "action_item": (
        "**针对你作为打野的角色**，在大龙刷新前90秒（18:30），清完上半野区后，"
        "优先购买真眼并放置在大龙坑上方河道草丛和蓝buff入口。"
        "这是打野的核心职责，即使延迟gank也要执行。"
    ),
    "expected_outcome": (
        "针对你持续的视野弱点，专注改进这一维度可以在未来10场比赛中"
        "将你的平均视野得分从45.2提升到至少60分，显著减少目标失误率。"
    ),
    "learning_resource": (
        "建议观看职业打野在赛前布置视野的 VOD（17:00-20:00时间段），"
        "对比你在这场比赛中的路径规划。"
    ),
    "priority": "critical",
}

print("🔬 Experiment 1: Profile Injection Impact")
print("\n" + "="*80)
print("Baseline (No Profile):")
print(f"  Issue: {baseline_suggestion['issue_identified']}")
print(f"  Action: {baseline_suggestion['action_item']}")
print(f"  Outcome: {baseline_suggestion['expected_outcome']}")

print("\n" + "="*80)
print("Personalized (With Profile):")
print(f"  Issue: {personalized_suggestion['issue_identified']}")
print(f"  Action: {personalized_suggestion['action_item']}")
print(f"  Outcome: {personalized_suggestion['expected_outcome']}")
print(f"  Learning: {personalized_suggestion['learning_resource']}")

print("\n" + "="*80)
print("📊 Analysis:")
print("  ✅ Personalized version acknowledges 20-match trend (45.2 avg)")
print("  ✅ Role-specific action (Jungle: clear camps → ward Baron)")
print("  ✅ Long-term outcome (improve from 45.2 → 60 over 10 matches)")
print("  ⚠️  Token cost: +87 tokens (+45% compared to baseline)")
print("  ⚠️  Risk: LLM must not hallucinate profile data")

### Experiment 2: Tone Customization for Different Player Types

In [ ]:
# Experiment 2: Mock LLM Responses for Different Tones

competitive_tone_suggestion = {
    "suggestion_id": "Vision_1456000",
    "dimension": "Vision",
    "issue_identified": "视野控制弱于队友（评分62.4，队伍排名4/5），导致24:16大龙失误。",
    "action_item": (
        "18:30-20:00时间窗口，强制执行大龙坑上方+蓝buff入口双真眼布置。"
        "优先级高于gank和补发育。数据显示职业打野在该时间段视野覆盖率≥80%。"
    ),
    "expected_outcome": "大龙视野覆盖率从0%→≥50%，降低被偷龙概率35%（历史数据）。",
    "priority": "critical",
}

casual_tone_suggestion = {
    "suggestion_id": "Vision_1456000",
    "dimension": "Vision",
    "issue_identified": (
        "这场比赛你的视野得分有点低（62.4分，队伍里排第4），"
        "24分16秒大龙被对面偷了，主要原因是咱们完全没视野。"
    ),
    "action_item": (
        "下次记得在大龙快刷新前（大概游戏进行到20分钟左右），"
        "花75块钱买个真眼放在大龙坑上面的草丛里。"
        "就算延迟一点回城买装备也没关系，视野更重要！"
        "可以把它想象成给队友买了个'保险'，防止对面偷龙翻盘。"
    ),
    "expected_outcome": (
        "有了这个真眼，队伍打大龙的时候就不会'两眼一抹黑'了，"
        "至少能看到对面来没来，大大降低被偷龙的风险。"
    ),
    "learning_resource": (
        "建议看看你的辅助队友这场比赛是怎么放眼的（17-24分钟时间段），"
        "他放了22个眼，平均每个眼存活87秒，可以学习一下！"
    ),
    "priority": "critical",
}

print("🔬 Experiment 2: Tone Customization")
print("\n" + "="*80)
print("Competitive Tone (竞技型玩家):")
print(f"  Issue: {competitive_tone_suggestion['issue_identified']}")
print(f"  Action: {competitive_tone_suggestion['action_item']}")
print(f"  Outcome: {competitive_tone_suggestion['expected_outcome']}")

print("\n" + "="*80)
print("Casual Tone (休闲型玩家):")
print(f"  Issue: {casual_tone_suggestion['issue_identified']}")
print(f"  Action: {casual_tone_suggestion['action_item']}")
print(f"  Outcome: {casual_tone_suggestion['expected_outcome']}")
print(f"  Learning: {casual_tone_suggestion['learning_resource']}")

print("\n" + "="*80)
print("📊 Analysis:")
print("  Competitive:")
print("    ✅ Concise, data-heavy (职业打野80%覆盖率)")
print("    ✅ Imperative tone (强制执行)")
print("    ✅ Technical jargon (时间窗口, 覆盖率)")
print("    📏 Token count: ~120 tokens")
print("  ")
print("  Casual:")
print("    ✅ Friendly, encouraging ('下次记得', '没关系')")
print("    ✅ Metaphors ('保险', '两眼一抹黑')")
print("    ✅ Explanatory (为什么75块钱, 为什么延迟回城)")
print("    📏 Token count: ~185 tokens (+54% compared to competitive)")
print("  ")
print("  💡 Recommendation: Maintain 2 tone variants (competitive/casual)")
print("     User classification can be inferred from match frequency & ranked tier")

### Experiment 3: Role-Specific Suggestion Variations

In [ ]:
# Experiment 3: Same Vision weakness, different roles → different suggestions

vision_suggestion_adc = {
    "role": "ADC",
    "action_item": (
        "作为ADC，在大龙刷新前（20分钟），优先购买1个真眼备用。"
        "在团队集结前往大龙区域时，如果辅助真眼用完，你应该主动补充放置在大龙坑入口。"
        "虽然这不是ADC的主要职责，但75块钱的投资可能决定5000金币的大龙buff归属。"
    ),
    "expected_outcome": (
        "确保团队在关键目标争夺时至少有2个真眼（辅助+你），"
        "弥补辅助可能的视野空窗期。"
    ),
}

vision_suggestion_jungle = {
    "role": "Jungle",
    "action_item": (
        "作为打野，你应该在18:30（大龙刷新前90秒）清完上半野区后，"
        "优先放置真眼在大龙坑上方河道草丛和敌方蓝buff入口。"
        "这是打野的核心职责，优先级高于继续刷野或gank。"
        "同时用饰品眼覆盖河道蟹区域，形成三角视野网。"
    ),
    "expected_outcome": (
        "建立大龙区域主动视野优势，提前发现敌方打野动向，"
        "使团队能够提前15-30秒集结或反蹲。"
    ),
}

vision_suggestion_support = {
    "role": "Support",
    "action_item": (
        "作为辅助，你应该在17:00就开始为大龙做视野准备："
        "(1) 17:00-18:00：放置真眼在大龙坑上方+河道草丛，消耗敌方扫描；"
        "(2) 18:30-19:00：补充第二轮真眼，确保至少1个存活到大龙刷新；"
        "(3) 19:30：携带2个真眼（1个备用）前往大龙区域。"
        "这是辅助的核心KPI，优先级最高。"
    ),
    "expected_outcome": (
        "确保团队在大龙刷新时拥有≥2个存活的真眼和饰品眼覆盖，"
        "建立视野控制优势，使团队能够主动开龙或反蹲敌方。"
    ),
}

print("🔬 Experiment 3: Role-Specific Suggestions")
print("\n" + "="*80)
print("Same Weakness (Vision), Different Roles:\n")

print("ADC:")
print(f"  Action: {vision_suggestion_adc['action_item']}")
print(f"  Outcome: {vision_suggestion_adc['expected_outcome']}")
print()

print("Jungle:")
print(f"  Action: {vision_suggestion_jungle['action_item']}")
print(f"  Outcome: {vision_suggestion_jungle['expected_outcome']}")
print()

print("Support:")
print(f"  Action: {vision_suggestion_support['action_item']}")
print(f"  Outcome: {vision_suggestion_support['expected_outcome']}")

print("\n" + "="*80)
print("📊 Analysis:")
print("  ✅ ADC: Defensive role (备用真眼, 补充辅助)")
print("  ✅ Jungle: Proactive role (18:30主动布置, 三角视野网)")
print("  ✅ Support: Primary responsibility (17:00开始, 分阶段计划)")
print("  ")
print("  💡 Recommendation: Inject 'preferred_role' into user profile")
print("     LLM can naturally generate role-appropriate suggestions")

---

## 📦 User Profile Data Contract

To enable V2.2 personalization, CLI 2 (Backend) must build and maintain user profiles.

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field


class V22UserProfile(BaseModel):
    """User profile for V2.2 personalization.
    
    This profile is built from user's historical match data (last 20 matches)
    and updated after each new analysis.
    """

    # User identification
    discord_user_id: str = Field(description="Discord user ID")
    puuid: str = Field(description="Riot PUUID")

    # Role & champion preferences
    preferred_role: Literal[
        "Top", "Jungle", "Mid", "ADC", "Support", "Fill"
    ] | None = Field(
        default=None,
        description=(
            "User's most frequently played role (≥40% of recent matches). "
            "Used for role-specific suggestions."
        ),
    )

    top_3_champions: list[str] = Field(
        default_factory=list,
        description="User's 3 most-played champions in last 20 matches",
        max_length=3,
    )

    # Performance trends (calculated from last 20 matches)
    avg_scores: dict[str, float] = Field(
        description=(
            "Average V1 scores across dimensions. "
            "Keys: 'Combat', 'Economy', 'Vision', 'Objective Control', 'Teamplay'"
        ),
        default_factory=dict,
    )

    persistent_weak_dimension: Literal[
        "Combat", "Economy", "Vision", "Objective Control", "Teamplay"
    ] | None = Field(
        default=None,
        description=(
            "Dimension where user scored below team average in ≥70% of matches. "
            "Used to prioritize suggestions."
        ),
    )

    # Player classification
    skill_level: Literal["beginner", "intermediate", "advanced"] = Field(
        default="intermediate",
        description=(
            "Skill level inferred from ranked tier: "
            "Iron/Bronze=beginner, Silver-Gold=intermediate, Plat+=advanced"
        ),
    )

    player_type: Literal["casual", "competitive"] = Field(
        default="casual",
        description=(
            "Player type inferred from match frequency: "
            "≥5 ranked matches/week + Gold+ tier = competitive, otherwise casual"
        ),
    )

    # Metadata
    total_matches_analyzed: int = Field(
        description="Total matches analyzed for this user", ge=0
    )
    last_updated: str = Field(
        description="Profile last update timestamp (ISO 8601 format)"
    )


# Example profile
example_profile = V22UserProfile(
    discord_user_id="123456789",
    puuid="test-puuid-abc123",
    preferred_role="Jungle",
    top_3_champions=["Lee Sin", "Graves", "Kha'Zix"],
    avg_scores={
        "Combat": 78.5,
        "Economy": 82.3,
        "Vision": 45.2,  # Persistent weakness
        "Objective Control": 71.8,
        "Teamplay": 68.9,
    },
    persistent_weak_dimension="Vision",
    skill_level="intermediate",
    player_type="competitive",
    total_matches_analyzed=23,
    last_updated="2025-10-06T15:30:00Z",
)

print("📦 V22UserProfile Contract:")
print(example_profile.model_dump_json(indent=2))

---

## 🎯 Recommended V2.2 Implementation Strategy

Based on the experiments above, here's the optimal approach:

### Phase 1: Profile Building (Week 1-2)

**Task**: CLI 2 implements `V22UserProfile` data model and profile builder.

**Profile Builder Logic**:
```python
async def build_user_profile(
    puuid: str,
    discord_user_id: str,
    recent_matches: list[MatchAnalysisResult],  # Last 20 V1 results
    ranked_tier: str,  # From Riot API
    match_frequency: float,  # Matches per week
) -> V22UserProfile:
    """
    Build user profile from historical data.
    """
    # Calculate preferred role (mode of recent_matches.role)
    # Calculate avg_scores (mean of each dimension)
    # Identify persistent_weak_dimension (dimension below team avg in ≥70% matches)
    # Infer skill_level from ranked_tier
    # Infer player_type from match_frequency + ranked_tier
    ...
```

**Database Schema Extension**:
```sql
CREATE TABLE user_profiles (
    discord_user_id TEXT PRIMARY KEY,
    puuid TEXT NOT NULL,
    profile_data JSONB NOT NULL,  -- V22UserProfile JSON
    last_updated TIMESTAMP NOT NULL,
    UNIQUE(puuid)
);

CREATE INDEX idx_user_profiles_puuid ON user_profiles(puuid);
```

---

### Phase 2: Hybrid Personalization (Week 3-4)

**Strategy**: Combine Approach A (prompt injection) + Approach C (tone variants)

**Why Hybrid**:
- **Prompt Injection**: Enables role-specific and trend-aware suggestions
- **Tone Variants**: Enables dramatic tone/complexity differences
- **Skip Approach B**: Post-processing can't change content, only ordering

**Implementation**:
```python
async def generate_prescriptive_analysis_v22(
    self,
    input_data: V21PrescriptiveAnalysisInput,
    user_profile: V22UserProfile,  # NEW
) -> V21PrescriptiveAnalysisReport:
    """
    V2.2: Personalized prescriptive analysis.
    """
    # 1. Select prompt template based on player_type
    if user_profile.player_type == "competitive":
        prompt_template = load_prompt("v22_coaching_competitive.txt")
    else:
        prompt_template = load_prompt("v22_coaching_casual.txt")
    
    # 2. Inject user profile context
    profile_context = generate_profile_context(user_profile, input_data)
    
    # 3. Format prompt
    formatted_prompt = prompt_template.format(
        user_profile_context=profile_context,  # NEW
        summoner_name=input_data.summoner_name,
        champion_name=input_data.champion_name,
        # ... other fields
    )
    
    # 4. Call LLM with JSON Mode
    # ... (same as V2.1)
```

**Profile Context Generator**:
```python
def generate_profile_context(
    user_profile: V22UserProfile,
    input_data: V21PrescriptiveAnalysisInput,
) -> str:
    """
    Generate user profile context for prompt injection.
    """
    context_parts = []
    
    # Role context
    if user_profile.preferred_role:
        context_parts.append(
            f"该用户是一个 {user_profile.preferred_role} 位置玩家"
        )
    
    # Persistent weakness context
    if user_profile.persistent_weak_dimension:
        dim = user_profile.persistent_weak_dimension
        avg_score = user_profile.avg_scores.get(dim, 0)
        context_parts.append(
            f"在最近20场比赛中，{dim} 维度得分持续偏低（平均 {avg_score:.1f} 分）"
        )
    
    # Champion context (if current champion is in top 3)
    if input_data.champion_name in user_profile.top_3_champions:
        context_parts.append(
            f"{input_data.champion_name} 是该用户的常用英雄之一"
        )
    
    return "**用户画像**: " + "，".join(context_parts) + "。"
```

---

### Phase 3: A/B Testing & Evaluation (Week 5-6)

**Variants**:
- **V2.1 (Baseline)**: No personalization
- **V2.2 (Personalized)**: With profile injection + tone customization

**Success Criteria**:
1. **Actionability**: V2.2 should maintain ≥75% actionability (same as V2.1)
2. **Helpfulness**: V2.2 should improve helpfulness rating by ≥5pp (e.g., 80% → 85%)
3. **Engagement**: Users should provide more detailed feedback on V2.2 suggestions
4. **Cost**: Token cost increase should be ≤30% (V2.1 baseline + 30%)

**Evaluation Metrics**:
- User feedback: `is_actionable`, `is_helpful` (from `V21SuggestionFeedback`)
- Engagement: Average `feedback_comment` length
- Cost: LLM token usage delta

---

## 📋 V2.2 Deliverables Summary

### For CLI 2 (Backend)

1. **Data Contract**: `src/contracts/v22_user_profile.py`
   - `V22UserProfile` model
   - Profile builder function

2. **Database Schema**: Migration to add `user_profiles` table

3. **Prompt Templates** (2 variants):
   - `src/prompts/v22_coaching_competitive.txt`
   - `src/prompts/v22_coaching_casual.txt`

4. **LLM Adapter Extension**: `generate_prescriptive_analysis_v22()` method

5. **A/B Test Configuration**: Enable V2.2 for 20% of users initially

### For CLI 1 (Frontend)

- **No changes required**: V2.2 output contract is identical to V2.1 (`V21PrescriptiveAnalysisReport`)
- Frontend rendering logic remains unchanged

---

## 🔮 Future Extensions (V2.3+)

### TTS (Text-to-Speech) Integration

**Hypothesis**: Casual players prefer audio feedback over reading text.

**Implementation**:
```python
# After generating V2.2 report, convert to audio
if user_profile.player_type == "casual" and user_preferences.enable_tts:
    tts_script = generate_tts_script(report)  # Simplify for speech
    audio_url = await tts_adapter.synthesize(
        text=tts_script,
        voice="zh-CN-Casual-Male",  # Friendly tone
    )
    # Attach audio_url to Discord response
```

**Challenges**:
- TTS script must be even more concise (remove technical jargon)
- Voice selection (male/female, age, tone)
- Cost (TTS API pricing)

---

### Champion-Specific Suggestions

**Hypothesis**: Vision suggestions for stealth champions (e.g., Evelynn, Shaco) should be different from traditional junglers.

**Example**:
- **Traditional Jungler (Lee Sin)**: "在18:30放置真眼在大龙坑"
- **Stealth Jungler (Evelynn)**: "利用你的隐身优势，在19:00潜入敌方野区放置真眼，同时侦查敌方打野位置"

**Implementation**: Inject champion archetype into profile context.

---

## ✅ Next Steps for CLI 4

1. **Share this research notebook** with CLI 2 (Backend) for V2.2 implementation planning
2. **Create engineering deliverables** (similar to V2.1):
   - `src/contracts/v22_user_profile.py`
   - `src/prompts/v22_coaching_competitive.txt`
   - `src/prompts/v22_coaching_casual.txt`
   - `docs/V2.2_ENGINEERING_INTEGRATION_GUIDE.md`
3. **Design V2.2 A/B test plan** with success criteria

---

**Research Status**: ✅ Complete  
**Ready for Engineering Handoff**: 🚧 Pending CLI 2 feedback on approach  
**Estimated V2.2 Implementation Time**: 4-6 weeks (profile building + personalization + A/B testing)
